In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings

# % matplotlib inline
plt.style.use('ggplot')

In [ ]:
pd.set_option('max_columns', 1000)
pd.set_option('max_row', 300)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df_train = pd.read_csv('train.csv')
df_train.columns


In [ ]:
df_train['SalePrice'].describe()

In [ ]:
sns.distplot(df_train['SalePrice'])

In [ ]:
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000))


In [ ]:
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000))

In [ ]:
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
fig = sns.boxplot(x=var, y='SalePrice', data=data)
fig.axis(ymin=0, ymax=800000)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000))

In [ ]:
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(20, 8))
fig = sns.boxplot(x=var, y='SalePrice', data=data)
fig.axis(ymin=0, ymax=800000)

In [ ]:
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, square=True, cmap='YlGnBu')
plt.show()


In [ ]:
cols = corrmat.nlargest(10, 'SalePrice').index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10},
                 yticklabels=cols.values, xticklabels=cols.values, cmap='YlGnBu')
plt.show()


In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
print(f"The train data size is: {train.shape}")
print(f"The test data size is: {test.shape}")
test



In [ ]:
train = train.drop(train.query('GrLivArea > 4000 and SalePrice < 300000').index)
fig, ax = plt.subplots()
ax.scatter(train['GrLivArea'], train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)


In [ ]:
from scipy import stats
from scipy.stats import norm

sns.distplot(train['SalePrice'], fit=stats.norm)

(mu, sigma) = norm.fit(train['SalePrice'])
print(f"mu = {mu} and sigma = {sigma}")
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma= ${:.2f}'.format(mu, sigma)], loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()



In [ ]:
train['SalePrice'] = np.log1p(train['SalePrice'])
sns.distplot(train.SalePrice, fit=norm)

mu, sigma = norm.fit(train.SalePrice)
print(f"mu={mu} sigma={sigma} ")

plt.legend([f'Normal dist. (\mu={mu} sigma={sigma}'])
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

fig = plt.figure()
res = stats.probplot(train.SalePrice, plot=plt)
plt.show()

y_train = train.SalePrice.values
n_train = train.shape[0]
n_test = test.shape[0]
all_data = pd.concat([train, test], ignore_index=True)
all_data.drop(['Id', 'SalePrice'], axis=1, inplace=True)
print(all_data.shape)

In [ ]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na[all_data_na > 0].sort_values(ascending=False)
all_data_na.head(100)
# all_data_na.index[:20]

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xticks(rotation=90)
plt.xlabel('Feature', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
plt.show()

In [ ]:
all_data.LotFrontage = all_data.groupby('Neighborhood')['LotFrontage'].apply(lambda x: x.fillna(x.median()))

In [ ]:
num_cols = ['MasVnrArea', 'BsmtUnfSF', 'TotalBsmtSF', 'GarageCars', 'BsmtFinSF2', 'BsmtFinSF1', 'GarageArea']
for col in num_cols:
    all_data[col].fillna(0, inplace=True)

In [ ]:
ob_cols = ["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageQual", "GarageCond", "GarageFinish",
           "GarageYrBlt", "GarageType", "BsmtExposure", "BsmtCond", "BsmtQual", "BsmtFinType2", "BsmtFinType1",
           "MasVnrType"]
for col in ob_cols:
    all_data[col].fillna('None', inplace=True)

In [ ]:
cat_cols = ["MSZoning", "BsmtFullBath", "BsmtHalfBath", "Utilities", "Functional", "Electrical", "KitchenQual",
            "SaleType", "Exterior1st", "Exterior2nd"]
for col in cat_cols:
    all_data[col].fillna(all_data[col].mode()[0], inplace=True)

In [ ]:
all_data.isnull().sum()[all_data.isnull().sum() > 0]


In [ ]:
NumStr = ["MSSubClass", "BsmtFullBath", "BsmtHalfBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr", "MoSold", "YrSold",
          "YearBuilt", "YearRemodAdd", "LowQualFinSF", "GarageYrBlt"]
for col in NumStr:
    all_data[col] = all_data[col].astype(str)


In [ ]:
all_data['Utilities'].value_counts()

In [ ]:
all_data = all_data.drop(['Utilities'], axis=1)


In [ ]:
all_data.shape



In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 'ExterQual', 'ExterCond', 'HeatingQC',
        'PoolQC', 'KitchenQual', 'BsmtFinType1', 'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish',
        'LandSlope', 'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 'YrSold',
        'MoSold', 'YearBuilt', 'YearRemodAdd', 'LowQualFinSF', 'GarageYrBlt']
lbl = LabelEncoder()
for c in cols:
    lbl.fit(list(all_data[c].values))
    all_data[c] = lbl.transform(list(all_data[c].values))


In [ ]:
all_data.shape


In [ ]:
from scipy.stats import norm, skew

numeric_feats = all_data.select_dtypes(exclude=['object']).columns
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print(' Skew in numerical features: \n')
skewness = pd.DataFrame({'Skew': skewed_feats})
skewness.head(10)


In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print(f'{skewness.shape[0]} skewed features')

from scipy.special import boxcox1p

skewed_feats = skewness.index
lam = 0.15
for feat in skewed_feats:
    all_data[feat] = boxcox1p(all_data[feat], lam)

In [ ]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

train = all_data[:n_train]
test = all_data[n_train:]

In [ ]:
def rmsle_cv(model):
    rmse = np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber')

In [ ]:
from sklearn.model_selection import GridSearchCV


class grid():
    def __init__(self, model):
        self.model = model

    def grid_set(self, X, y, param_grid):
        grid_search = GridSearchCV(self.model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X, y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params', 'mean_test_score', 'std_test_score']])

In [ ]:
grid(Lasso()).grid_set(train, y_train, {'alpha': [0.0004, 0.0005, 0.0007, 0.0003, 0.0009, 0.0008], 'max_iter': [10000]})

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        print("init")
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.base_models_ = None
        self.meta_model_ = None

    def fit(self, X, y):
        print("fit")
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            print(f"progress: {i} /{len(self.base_models)}")
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                print("_", end="")
            print()
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    def predict(self, X):
        print("predict")
        print("")
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_])
        return self.meta_model_.predict(meta_features)




In [ ]:
# stacked_averaged_models = StackingAveragedModels(base_models=(ENet, GBoost, KRR), meta_model=lasso)
# score = rmsle_cv(stacked_averaged_models)
# print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models=(ENet, GBoost, KRR), meta_model=lasso)
stacked_averaged_models.fit(train.values,y_train)


In [ ]:
test1 = pd.read_csv('test.csv')
pred = np.exp(stacked_averaged_models.predict(test))
result = pd.DataFrame({'Id': test1.Id, 'SalePrice': pred})
result.to_csv("submission.csv", index=False)
result
